# Práctica IDS con Suricata (ejecución remota en Jupyter)

En esta práctica aprenderás a **instalar, configurar y ejecutar Suricata** como motor IDS para:
- **Captura en vivo** (tráfico de una interfaz).
- **Análisis offline** de ficheros **PCAP**.
- **Inspección de logs** (`fast.log` y `eve.json`) para interpretar alertas y eventos.
<a href="https://colab.research.google.com/github/UPM-RSTI/RTVE/blob/ef8d599044a8f1e0ea80b04e7280ffe224b5da55/SURI_IDS_MASTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# Abrir en Google Colab
> **Importante:** ejecuta las celdas **en orden**.


## Introducción: Suricata y los IDS

**Suricata** es un motor IDS/IPS de código abierto y alto rendimiento. Permite:
- Inspección profunda de paquetes (DPI) en tiempo real.
- Detección por firmas (reglas) y generación de alertas.
- Registro estructurado de eventos (por ejemplo en `eve.json`).

Un **IDS (Intrusion Detection System)** analiza tráfico o eventos para detectar actividad maliciosa o no autorizada. En esta práctica nos centramos en **detección y análisis**, interpretando las alertas que produce Suricata.


## Requisitos y recomendaciones

**Entorno recomendado**
- Linux (Ubuntu/Debian) con permisos de `sudo`.
- Conexión a Internet para instalar paquetes y descargar PCAPs.

**Antes de empezar**
- Si estás en un entorno gestionado (p. ej., un servidor remoto o una VM), asegúrate de tener:
  - Acceso a `sudo`.
  - Espacio libre suficiente (varios cientos de MB para reglas y PCAPs).
  - Permiso para capturar tráfico (solo necesario para el modo en vivo).

**Convención de rutas en esta práctica**
- Configuración principal: `/etc/suricata/suricata.yaml`
- Reglas actualizadas (habitual tras `suricata-update`): `/var/lib/suricata/rules/suricata.rules`
- Logs típicos: `/var/log/suricata/` (aunque puede variar si se usa `-l` al ejecutar Suricata)


# Práctica IDS – Suricata

Sigue los pasos en orden. Cuando veas un bloque con comandos, ejecútalo y revisa la salida antes de continuar.

In [ ]:
!sudo apt-get update
!sudo apt-get install -y suricata

## 1. Captura de tráfico en vivo (opcional)

En este apartado ejecutarás Suricata escuchando una **interfaz de red** y generando logs en tiempo real.

1) Lista las interfaces disponibles y elige la que corresponda a tu conexión (por ejemplo `eth0`, `enp0s3`, `wlan0`).  
2) Ejecuta Suricata en modo captura sobre esa interfaz.

> Si tu entorno **no permite captura en vivo** (p. ej., restricciones de contenedor o permisos), pasa al apartado **Análisis de PCAP**.


In [ ]:
# Ejecuta este comando para listar tus interfaces de red y encontrar el nombre de la interfaz que deseas monitorear (por ejemplo, eth0, enp0s3, wlan0).
!ip a

Una vez que tengas el nombre de la interfaz (por ejemplo, `eth0`), inicia Suricata en modo captura.

- Este comando suele ejecutarse en primer plano.
- Para detenerlo, usa `Ctrl+C`.

> **Sugerencia:** si quieres ver rápidamente si se están generando alertas, revisa después los logs (`/var/log/suricata/fast.log` y `/var/log/suricata/eve.json`).


In [ ]:
# Reemplaza '<nombre_de_interfaz>' con tu interfaz real (e.g., eth0, enp0s3).
# Este comando requiere permisos de superusuario.
# !sudo suricata -i <nombre_de_interfaz> -c /etc/suricata/suricata.yaml -s /etc/suricata/rules/suricata.rules

# Ejemplo (descomenta y adapta si conoces tu interfaz):
#!sudo suricata -i eth0 -c /etc/suricata/suricata.yaml -s /etc/suricata/rules/suricata.rules

## 2. Revisar la configuración de Suricata (`suricata.yaml`)

Antes de analizar tráfico, revisaremos el fichero de configuración para entender:
- Dónde busca Suricata las **reglas**.
- Cómo están definidas las variables de red (`HOME_NET`, `EXTERNAL_NET`).
- Qué **salidas de log** están habilitadas (especialmente `fast.log` y `eve.json`).


In [ ]:
print('Contenido de /etc/suricata/suricata.yaml:')
!sudo cat /etc/suricata/suricata.yaml

## 3. Puntos clave en `suricata.yaml` (guía rápida)

En la práctica nos fijaremos especialmente en:
- **Ruta de reglas** (`default-rule-path` y `rule-files`)
- **Variables de red** (`HOME_NET`, `EXTERNAL_NET`)
- **Grupos de puertos** (`port-groups`)
- **Logs de salida** (`outputs`: `eve.json`, `fast.log`)

A continuación tienes una explicación ampliada para ayudarte a interpretar la configuración:

 de Atributos Importantes en `suricata.yaml`

El archivo `suricata.yaml` es el corazón de la configuración de Suricata. A continuación, se explican algunos de los atributos clave, con énfasis en el fichero de reglas, direcciones IP, puertos, `HOME_NET` y `EXTERNAL_NET`.

### 1. `default-rule-path`

*   **Ubicación:** Generalmente se encuentra bajo la sección `rules:`. Establece la ruta base donde Suricata buscará los archivos de reglas.
*   **Ejemplo:** `default-rule-path: /etc/suricata/rules`
*   **Importancia:** Define el directorio principal para tus reglas. `suricata-update` suele poner las reglas compiladas en `/var/lib/suricata/rules/`.

### 2. `rule-files`

*   **Ubicación:** También bajo la sección `rules:`, especifica qué archivos de reglas individuales debe cargar Suricata.
*   **Ejemplo:**
    ```yaml
    rule-files:
      - suricata.rules
      # - app-layer-events.rules
    ```
*   **Importancia:** Aquí es donde le dices a Suricata qué colecciones de reglas usar. Como se vio anteriormente, si `suricata-update` coloca las reglas en `/var/lib/suricata/rules/suricata.rules`, es crucial que esta sección apunte a esa ubicación o que el archivo `suricata.rules` en `/etc/suricata/rules/` contenga una referencia correcta.

### 3. Configuración de Red: `HOME_NET` y `EXTERNAL_NET`

Estos son dos de los parámetros más críticos en Suricata, ya que definen qué se considera tu red interna (protegida) y qué es externo a ella. Las reglas de Suricata a menudo utilizan estas variables para distinguir entre tráfico interno y externo, lo que es fundamental para la detección precisa.

*   **`HOME_NET`**
    *   **Ubicación:** Bajo la sección `vars:`. Define el rango de direcciones IP de tu red interna o la red que deseas proteger.
    *   **Ejemplo:** `HOME_NET: "[192.168.0.0/16,10.0.0.0/8,172.16.0.0/12]"`
    *   **Importancia:** Todas las direcciones IP que se encuentran dentro de este rango son consideradas 'locales' o 'internas'. Las reglas de Suricata interpretan esto para distinguir los ataques que vienen de fuera (externos a internos) de los internos (internos a internos o internos a externos).

*   **`EXTERNAL_NET`**
    *   **Ubicación:** Bajo la sección `vars:`. Define todas las direcciones IP que no son parte de `HOME_NET` (generalmente, Internet o cualquier otra red fuera de tu control).
    *   **Ejemplo:** `EXTERNAL_NET: "!$HOME_NET"`
    *   **Importancia:** `!$HOME_NET` es la forma más común y segura de configurarlo, lo que significa "todo lo que NO está en HOME_NET". Esto asegura que todo el tráfico que no es interno sea considerado externo, simplificando la escritura y el mantenimiento de las reglas.

### 4. `port-groups`

*   **Ubicación:** Bajo la sección `vars:`. Permite definir grupos de puertos comúnmente utilizados para diferentes servicios.
*   **Ejemplo:**
    ```yaml
    vars:
      # port groups
      HTTP_PORTS: "80,8080,8000,8008,8888,8443,443"
      FILE_DATA_PORTS: "$HTTP_PORTS,110,143,443,465,993,995,20,21,25,3306,5432,5900,8000,8080,8443,8888"
    ```
*   **Importancia:** Estas variables se utilizan en las reglas para que no tengas que enumerar todos los puertos en cada regla, haciendo las reglas más legibles y fáciles de mantener. Por ejemplo, una regla para tráfico HTTP puede simplemente usar `$HTTP_PORTS` en lugar de `80,8080,...`.

### 5. `outputs` (Salidas de Logs)

*   **Ubicación:** Una sección principal en el archivo `suricata.yaml`.
*   **Ejemplo (relevante):**
    ```yaml
    outputs:
      - eve-log:
          enabled: yes
          filetype: regular #regular|syslog|unix_dgram|unix_stream
          filename: eve.json
          # Appending to an existing file can be done by setting 'append' to 'yes'.
          # append: no
          # If you want to dump alerts to a separate file, you can do this:
          # - alert:
          #     enabled: yes
          #     # filetype: regular # regular|syslog|unix_dgram|unix_stream
          #     # filename: alerts.json
          #     # append: yes
      - fast.log:
          enabled: yes
          filetype: regular
          # By default, messages are appended to fast.log.
          # Change to 'no' to overwrite at startup.
          # append: yes
          filename: fast.log
          # The fast log format: tag, timestamp, alert info
          # If you want to specify a specific format for the fast log, you can use:
          # format: "%T.%{usec} %F %{%H:%M:%S} %{%Y-%m-%d} %i %s %p %d %H %M %{alert.signature} %{alert.gid} %{alert.rev} %{alert.signature_id} %{event_type} %{src_ip} %{dest_ip} %{src_port} %{dest_port} %{proto}"
    ```
*   **Importancia:** Esta sección configura dónde y cómo Suricata guarda sus logs. Hemos configurado `eve.json` para eventos detallados (incluidas alertas) y `fast.log` para alertas concisas. La ruta donde se escriben estos archivos de log puede ser sobrescrita con la opción `-l` al ejecutar Suricata (como hicimos con `/tmp/suricata_logs`).

Comprender y ajustar estas configuraciones es fundamental para un monitoreo de red eficaz con Suricata.

## 4. Análisis de ficheros PCAP

Ahora ejecutaremos Suricata en modo **offline** para analizar un PCAP ya capturado.

- Este modo es ideal para prácticas remotas, porque no requiere capturar tráfico en vivo.
- Tras el análisis, revisaremos los logs para identificar alertas y eventos relevantes.


In [ ]:
# Reemplaza '<ruta_al_archivo_pcap>' con la ruta a tu archivo PCAP.
# Este comando requiere permisos de superusuario.
# !sudo suricata -r <ruta_al_archivo_pcap> -c /etc/suricata/suricata.yaml -s /etc/suricata/rules/suricata.rules

# Ejemplo (descomenta y adapta si tienes un archivo PCAP):
# !sudo suricata -r /home/user/mi_captura.pcap -c /etc/suricata/suricata.yaml -s /etc/suricata/rules/suricata.rules

## 5. Visualización de logs de Suricata

Suricata suele generar, como mínimo, dos ficheros útiles:

- `fast.log`: listado **resumido** de alertas (rápido de inspeccionar).
- `eve.json`: eventos en **JSON** (más detallado; útil para búsquedas, filtrado y parsing).

En las siguientes celdas mostraremos ambos.
> Si los ficheros no aparecen en el directorio actual, revisa también `/var/log/suricata/`.


In [ ]:
# Muestra el contenido del log fast.log
!sudo cat /var/log/suricata/fast.log

In [ ]:
# Muestra las últimas líneas del log eve.json en tiempo real (requiere que Suricata esté corriendo y generando logs)
!sudo cat /var/log/suricata/eve.json

## 6. Descarga y descompresión de un PCAP de ejemplo

Vamos a descargar un PCAP comprimido con contraseña (caso real publicado para análisis).
- Descarga el `.zip`
- Descomprímelo con la contraseña indicada
- Verifica que el `.pcap` se ha extraído correctamente


## 7. Actualización de reglas

Antes de analizar tráfico, actualizamos las reglas para maximizar la capacidad de detección.

- `suricata-update` descarga y compila reglas en una ruta que suele ser:  
  `/var/lib/suricata/rules/suricata.rules`

> Si Suricata no encuentra reglas o muestra avisos de rutas, revisa el apartado de configuración de reglas en `suricata.yaml`.


In [ ]:
!sudo suricata-update

In [ ]:
# Descargar el archivo .pcap.zip
!wget https://www.malware-traffic-analysis.net/2024/11/24/2024-11-24-webserver-scans-and-probes.pcap.zip

# Descomprimir el archivo para obtener el .pcap con la contraseña
!unzip -P infected_20241124 2024-11-24-webserver-scans-and-probes.pcap.zip

# Listar los archivos para confirmar que el .pcap fue extraído
!ls -l

In [ ]:
import subprocess

# Realiza una copia de seguridad del archivo suricata.yaml original antes de modificarlo
!sudo cp /etc/suricata/suricata.yaml /etc/suricata/suricata.yaml.bak

# Usa sed para reemplazar la ruta de reglas predeterminada con la actualizada.
# Este comando asume que la regla predeterminada se especifica como '- suricata.rules'
# y que se encuentra dentro del directorio de reglas especificado por 'default-rule-path',
# que generalmente es '/etc/suricata/rules/'.
# Reemplazaremos la línea que apunta al archivo de reglas antiguo por la nueva.

# Primero, identifica la línea a reemplazar. Normalmente es una línea como '- suricata.rules'
# bajo la sección 'rule-files:'.
# La herramienta suricata-update crea un único 'suricata.rules' en /var/lib/suricata/rules/
# Por lo tanto, necesitamos comentar cualquier entrada de archivo de reglas existente y añadir la nueva.

# Lee el contenido original del archivo
with open('/etc/suricata/suricata.yaml', 'r') as f:
    lines = f.readlines()

new_lines = []
rule_files_section = False
inserted_new_rule = False

for line in lines:
    if 'rule-files:' in line:
        new_lines.append(line) # Mantiene la línea 'rule-files:'
        rule_files_section = True
        # Añade la ruta de reglas actualizada inmediatamente después de la declaración 'rule-files:'
        indent = len(line) - len(line.lstrip()) # Preserva la indentación
        new_lines.append(f'{' ' * indent}- /var/lib/suricata/rules/suricata.rules\n')
        inserted_new_rule = True
    elif rule_files_section and line.strip().startswith('- '):
        # Comenta las entradas de archivos de reglas antiguas si están en la ruta predeterminada
        if '/etc/suricata/rules/' in line or 'suricata.rules' in line:
            new_lines.append(f'# {line}')
        else:
            new_lines.append(line) # Mantiene otros archivos de reglas que puedan ser válidos
    elif rule_files_section and not line.strip(): # Fin de la sección rule-files (línea vacía)
        rule_files_section = False
        new_lines.append(line)
    else:
        new_lines.append(line)

# Si la sección rule-files no se encontró pero necesitamos añadir la regla
if not inserted_new_rule:
    # Este caso podría ser más complejo si la sección 'rule-files' no existe en absoluto o está estructurada de manera diferente.
    # Para esta solución específica, asumimos que 'rule-files' existe y lo estamos modificando.
    # Si no existiera, necesitaríamos insertarlo estratégicamente.
    # Para simplificar, si no se encuentra y no se inserta, asumimos que ya está manejado o no es relevante.
    # En un escenario real, podría ser necesario un análisis más robusto.
    pass # No se necesita ninguna acción si la regla se insertó correctamente arriba o si no hay sección rule-files para empezar

# Escribe el contenido modificado de nuevo en suricata.yaml
with open('/etc/suricata/suricata.yaml', 'w') as f:
    f.writelines(new_lines)

print("suricata.yaml actualizado para usar reglas de /var/lib/suricata/rules/suricata.rules")

In [ ]:
import os

pcap_file_path = '/content/2024-11-24-webserver-scans-and-probes.pcap'
config_file_path = '/etc/suricata/suricata.yaml'
rules_file_path = '/var/lib/suricata/rules/suricata.rules' # Esta ruta ahora también está configurada en suricata.yaml

# Asegurarse de que el archivo PCAP existe antes de intentar analizarlo
if not os.path.exists(pcap_file_path):
    print(f"Error: Archivo PCAP no encontrado en {pcap_file_path}")
else:
    print(f"Re-analizando el archivo PCAP: {pcap_file_path} con la configuración actualizada...")
    !sudo suricata -r {pcap_file_path} -c {config_file_path} --runmode autofp

# Nota: Ahora podríamos omitir el argumento -s ya que la ruta de las reglas está establecida en el archivo de configuración,
# pero mantenerlo explícitamente aquí también es inofensivo ya que sobrescribe la configuración si es diferente.
# Sin embargo, para probar verdaderamente la modificación del archivo de configuración, podemos eliminar -s.
# Vamos a eliminar -s para confirmar que el cambio en el archivo de configuración tuvo efecto para la carga de reglas.
# Re-ejecutando sin -s para depender únicamente del suricata.yaml actualizado para la ruta de las reglas.
print("Re-analizando el archivo PCAP de nuevo, confiando en el suricata.yaml actualizado para la ruta de las reglas...")
!sudo suricata -r {pcap_file_path} -c {config_file_path} --runmode autofp

In [ ]:
print("Contenido de fast.log después del análisis de WannaCry:")
!sudo cat fast.log

In [ ]:
print("Contenido de eve.json después del análisis de WannaCry:")
!sudo cat eve.json

## 8. Caso de estudio: WannaCry / EternalBlue (PCAP histórico)

En este apartado analizaremos un PCAP histórico asociado al exploit **EternalBlue** (MS17-010) usado en campañas como WannaCry.

Objetivo:
- Ejecutar Suricata sobre el PCAP.
- Inspeccionar `fast.log` y `eve.json` buscando alertas relevantes.


In [ ]:
pcap_zip_url = 'https://www.malware-traffic-analysis.net/2017/05/18/2017-05-18-WannaCry-ransomware-using-EnternalBlue-exploit.pcap.zip'
pcap_password = 'infected_20170518'

# Descargar el archivo .pcap.zip
print(f"Descargando el archivo: {pcap_zip_url}")
!wget {pcap_zip_url}

# Descomprimir el archivo para obtener el .pcap con la contraseña
pcap_zip_filename = pcap_zip_url.split('/')[-1]
print(f"Descomprimiendo el archivo: {pcap_zip_filename} con contraseña...")
!unzip -P {pcap_password} {pcap_zip_filename}

# Listar los archivos para confirmar que el .pcap fue extraído
print("Listando archivos en el directorio actual:")
!ls -l

In [ ]:
import os

pcap_file_path = '/content/2017-05-18-WannaCry-ransomware-using-EnternalBlue-exploit.pcap'
config_file_path = '/etc/suricata/suricata.yaml'
rules_file_path = '/var/lib/suricata/rules/suricata.rules' # Esta ruta también está configurada ahora en suricata.yaml

# Asegúrate de que el archivo PCAP existe antes de intentar analizarlo
if not os.path.exists(pcap_file_path):
    print(f"Error: Archivo PCAP no encontrado en {pcap_file_path}")
else:
    print(f"Re-analizando el archivo PCAP: {pcap_file_path} con la configuración actualizada...")
    !sudo suricata -r {pcap_file_path} -c {config_file_path} --runmode autofp

# Nota: Ahora podríamos omitir el argumento -s ya que la ruta de las reglas está establecida en el archivo de configuración,
# pero mantenerlo explícitamente aquí también es inofensivo ya que sobrescribe la configuración si es diferente.
# Sin embargo, para probar verdaderamente la modificación del archivo de configuración, podemos eliminar -s. Vamos a eliminar -s
# para confirmar que el cambio en el archivo de configuración tuvo efecto para la carga de reglas.
print("Re-analizando el archivo PCAP de nuevo, confiando en el suricata.yaml actualizado para la ruta de las reglas...")
!sudo suricata -r {pcap_file_path} -c {config_file_path} --runmode autofp

Re-analyzing PCAP file: /content/2017-05-18-WannaCry-ransomware-using-EnternalBlue-exploit.pcap with updated configuration...
26/2/2026 -- 10:29:31 - <Notice> - This is Suricata version 6.0.4 RELEASE running in USER mode
26/2/2026 -- 10:31:49 - <Notice> - all 3 packet processing threads, 4 management threads initialized, engine started.
26/2/2026 -- 10:31:50 - <Notice> - Signal Received.  Stopping engine.
26/2/2026 -- 10:31:50 - <Notice> - Pcap-file module read 1 files, 46654 packets, 37044839 bytes
Re-analyzing PCAP file again, relying on updated suricata.yaml for rules path...
26/2/2026 -- 10:31:51 - <Notice> - This is Suricata version 6.0.4 RELEASE running in USER mode
26/2/2026 -- 10:34:05 - <Notice> - all 3 packet processing threads, 4 management threads initialized, engine started.
26/2/2026 -- 10:34:06 - <Notice> - Signal Received.  Stopping engine.
26/2/2026 -- 10:34:06 - <Notice> - Pcap-file module read 1 files, 46654 packets, 37044839 bytes


In [ ]:
print("Contenido de fast.log después del análisis de WannaCry:")
!sudo cat fast.log

In [ ]:
print("Contenido de eve.json después del análisis de WannaCry:")
!sudo cat eve.json